In [4]:
import requests
import pandas as pd
import json
import numpy as np

# Get Weather Data


Geo coords for Dhaka, Chittagong, and Patuakhali


In [5]:
# tuple as (lat, lon)
dhk_coords = (23.7104, 90.40744)
cht_coords = (22.3384, 91.83168)
pat_coords = (22.36833, 90.3458)

In [6]:
def get_city_weather(coords):
    weather_api_url = f"https://api.open-meteo.com/v1/forecast?latitude={dhk_coords[0]}&longitude={dhk_coords[1]}&daily=relative_humidity_2m_mean,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum,sunshine_duration&forecast_days=7"
    response = requests.get(weather_api_url)

    try:
        response.raise_for_status()
        data = response.json()

        return data
    except requests.exceptions.RequestException as e:
        print(e)

In [7]:
dhaka_data = get_city_weather(dhk_coords)
dhaka_data

{'latitude': 23.75,
 'longitude': 90.375,
 'generationtime_ms': 0.05364418029785156,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 27.0,
 'daily_units': {'time': 'iso8601',
  'relative_humidity_2m_mean': '%',
  'temperature_2m_max': '°C',
  'temperature_2m_min': '°C',
  'temperature_2m_mean': '°C',
  'rain_sum': 'mm',
  'sunshine_duration': 's'},
 'daily': {'time': ['2025-08-26',
   '2025-08-27',
   '2025-08-28',
   '2025-08-29',
   '2025-08-30',
   '2025-08-31',
   '2025-09-01'],
  'relative_humidity_2m_mean': [77, 79, 83, 84, 84, 78, 79],
  'temperature_2m_max': [34.1, 33.1, 31.0, 31.2, 31.4, 33.6, 33.9],
  'temperature_2m_min': [26.4, 25.8, 25.8, 25.8, 25.7, 25.9, 26.5],
  'temperature_2m_mean': [30.0, 28.8, 28.0, 28.1, 28.3, 29.6, 29.8],
  'rain_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'sunshine_duration': [43170.99,
   35086.55,
   29355.06,
   36263.34,
   37452.39,
   40652.68,
   37738.73]}}

In [8]:
city_weather_df = pd.DataFrame(
    columns=[
        "Date",  # Remove Date before passed to model
        "Rainfall",
        "Sunshine",
        "Humidity",
        "Temp_mean",
        "Temp_max",
        "Temp_min",
        "Year",
        "Month",
        "loadshed_prev",
        "generation_prev",
    ]
)
city_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev


In [9]:
dhaka_data["daily"]

{'time': ['2025-08-26',
  '2025-08-27',
  '2025-08-28',
  '2025-08-29',
  '2025-08-30',
  '2025-08-31',
  '2025-09-01'],
 'relative_humidity_2m_mean': [77, 79, 83, 84, 84, 78, 79],
 'temperature_2m_max': [34.1, 33.1, 31.0, 31.2, 31.4, 33.6, 33.9],
 'temperature_2m_min': [26.4, 25.8, 25.8, 25.8, 25.7, 25.9, 26.5],
 'temperature_2m_mean': [30.0, 28.8, 28.0, 28.1, 28.3, 29.6, 29.8],
 'rain_sum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'sunshine_duration': [43170.99,
  35086.55,
  29355.06,
  36263.34,
  37452.39,
  40652.68,
  37738.73]}

In [10]:
daily_data = dhaka_data["daily"]

city_weather_df["Date"] = daily_data["time"]
city_weather_df["Date"] = pd.to_datetime(city_weather_df["Date"], format="%Y-%m-%d")
city_weather_df["Year"] = city_weather_df["Date"].dt.year
city_weather_df["Month"] = city_weather_df["Date"].dt.month

city_weather_df["Rainfall"] = daily_data["rain_sum"]
city_weather_df["Sunshine"] = daily_data["sunshine_duration"]
city_weather_df["Sunshine"] = city_weather_df["Sunshine"] / (60 * 60)
city_weather_df["Humidity"] = daily_data["relative_humidity_2m_mean"]

city_weather_df["Temp_mean"] = daily_data["temperature_2m_mean"]
city_weather_df["Temp_max"] = daily_data["temperature_2m_max"]
city_weather_df["Temp_min"] = daily_data["temperature_2m_min"]

city_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77,30.0,34.1,26.4,2025,8,NaN,NaN
1,2025-08-27,0.0,9.746264,79,28.8,33.1,25.8,2025,8,NaN,NaN
2,2025-08-28,0.0,8.154183,83,28.0,31.0,25.8,2025,8,NaN,NaN
3,2025-08-29,0.0,10.073150,84,28.1,31.2,25.8,2025,8,NaN,NaN
4,2025-08-30,0.0,10.403442,84,28.3,31.4,25.7,2025,8,NaN,NaN
5,2025-08-31,0.0,11.292411,78,29.6,33.6,25.9,2025,8,NaN,NaN
6,2025-09-01,0.0,10.482981,79,29.8,33.9,26.5,2025,9,NaN,NaN


In [11]:
def get_city_weather_df(daily_data):
    city_df = pd.DataFrame(
        columns=[
            "Date",  # Remove Date before passed to model
            "Rainfall",
            "Sunshine",
            "Humidity",
            "Temp_mean",
            "Temp_max",
            "Temp_min",
            "Year",
            "Month",
            "loadshed_prev",
            "generation_prev",
        ]
    )

    city_df["Date"] = daily_data["time"]
    city_df["Date"] = pd.to_datetime(city_df["Date"], format="%Y-%m-%d")
    city_df["Year"] = city_df["Date"].dt.year
    city_df["Month"] = city_df["Date"].dt.month

    city_df["Rainfall"] = daily_data["rain_sum"]
    city_df["Sunshine"] = daily_data["sunshine_duration"]
    # Convert seconds to hours
    city_df["Sunshine"] = city_df["Sunshine"] / (60 * 60)
    city_df["Humidity"] = daily_data["relative_humidity_2m_mean"]

    city_df["Temp_mean"] = daily_data["temperature_2m_mean"]
    city_df["Temp_max"] = daily_data["temperature_2m_max"]
    city_df["Temp_min"] = daily_data["temperature_2m_min"]

    return city_df

## Get weather data of all locations


In [13]:
dhk_data = get_city_weather(dhk_coords)
dhk_weather_df = get_city_weather_df(dhk_data["daily"])
dhk_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77,30.0,34.1,26.4,2025,8,NaN,NaN
1,2025-08-27,0.0,9.746264,79,28.8,33.1,25.8,2025,8,NaN,NaN
2,2025-08-28,0.0,8.154183,83,28.0,31.0,25.8,2025,8,NaN,NaN
3,2025-08-29,0.0,10.073150,84,28.1,31.2,25.8,2025,8,NaN,NaN
4,2025-08-30,0.0,10.403442,84,28.3,31.4,25.7,2025,8,NaN,NaN
5,2025-08-31,0.0,11.292411,78,29.6,33.6,25.9,2025,8,NaN,NaN
6,2025-09-01,0.0,10.482981,79,29.8,33.9,26.5,2025,9,NaN,NaN


In [14]:
cht_data = get_city_weather(cht_coords)
cht_weather_df = get_city_weather_df(cht_data["daily"])
cht_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77,30.0,34.1,26.4,2025,8,NaN,NaN
1,2025-08-27,0.0,9.746264,79,28.8,33.1,25.8,2025,8,NaN,NaN
2,2025-08-28,0.0,8.154183,83,28.0,31.0,25.8,2025,8,NaN,NaN
3,2025-08-29,0.0,10.073150,84,28.1,31.2,25.8,2025,8,NaN,NaN
4,2025-08-30,0.0,10.403442,84,28.3,31.4,25.7,2025,8,NaN,NaN
5,2025-08-31,0.0,11.292411,78,29.6,33.6,25.9,2025,8,NaN,NaN
6,2025-09-01,0.0,10.482981,79,29.8,33.9,26.5,2025,9,NaN,NaN


In [15]:
pat_data = get_city_weather(pat_coords)
pat_weather_df = get_city_weather_df(pat_data["daily"])
pat_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77,30.0,34.1,26.4,2025,8,NaN,NaN
1,2025-08-27,0.0,9.746264,79,28.8,33.1,25.8,2025,8,NaN,NaN
2,2025-08-28,0.0,8.154183,83,28.0,31.0,25.8,2025,8,NaN,NaN
3,2025-08-29,0.0,10.073150,84,28.1,31.2,25.8,2025,8,NaN,NaN
4,2025-08-30,0.0,10.403442,84,28.3,31.4,25.7,2025,8,NaN,NaN
5,2025-08-31,0.0,11.292411,78,29.6,33.6,25.9,2025,8,NaN,NaN
6,2025-09-01,0.0,10.482981,79,29.8,33.9,26.5,2025,9,NaN,NaN


In [16]:
all_weather_df = pd.concat([dhk_weather_df, cht_weather_df, pat_weather_df])

all_weather_df = (
    all_weather_df.groupby("Date")
    .agg(
        {
            "Rainfall": "mean",
            "Sunshine": "mean",
            "Humidity": "mean",
            "Temp_mean": "mean",
            "Temp_max": "max",
            "Temp_min": "min",
            "Year": "first",
            "Month": "first",
            "loadshed_prev": "first",
            "generation_prev": "first",
        }
    )
    .reset_index()
)

all_weather_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77.0,30.0,34.1,26.4,2025,8,None,None
1,2025-08-27,0.0,9.746264,79.0,28.8,33.1,25.8,2025,8,None,None
2,2025-08-28,0.0,8.154183,83.0,28.0,31.0,25.8,2025,8,None,None
3,2025-08-29,0.0,10.073150,84.0,28.1,31.2,25.8,2025,8,None,None
4,2025-08-30,0.0,10.403442,84.0,28.3,31.4,25.7,2025,8,None,None
5,2025-08-31,0.0,11.292411,78.0,29.6,33.6,25.9,2025,8,None,None
6,2025-09-01,0.0,10.482981,79.0,29.8,33.9,26.5,2025,9,None,None


# Get Previous Day's Electricity Data


In [ ]:
from datetime import datetime, timedelta

yesterday_date = datetime.today() - timedelta(days=1)

yesterday_date = yesterday_date.strftime("%d-%m-%Y")
yesterday_date

'26-08-2025'

In [18]:
import io

pgcb_url = f"https://erp.pgcb.gov.bd/w/generations/view_generations?page={1}"

try:
    pgcb_res = requests.get(pgcb_url, verify=False)
    pgcb_res.raise_for_status()
    html = pgcb_res.text

    tables = pd.read_html(io.StringIO(html))

    electricity_df = tables[0]

    electricity_df.columns = electricity_df.columns.droplevel(0)
    electricity_df = electricity_df[
        ["Date", "Generation(MW)", "Demand(MW)", "Loadshed"]
    ].copy()

    electricity_df = electricity_df[electricity_df["Date"] == yesterday_date]

    electricity_df.rename(
        columns={
            "Generation(MW)": "Generation",
            "Demand(MW)": "Demand",
        },
        inplace=True,
    )

    # Convert units from MW to GW
    electricity_df[["Generation", "Demand", "Loadshed"]] = (
        electricity_df[["Generation", "Demand", "Loadshed"]] / 1000
    )
except requests.exceptions.RequestException as e:
    print(e)

/home/mashrur/anaconda3/envs/my_env/lib/python3.12/site-packages/urllib3/connectionpool.py:2303: InsecureRequestWarning: Unverified HTTPS request is being made to host 'erp.pgcb.gov.bd'. Adding certificate verification is strongly advised. See: https://urllib3future.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
electricity_df

,Date,Generation,Demand,Loadshed
0,26-08-2025,15.748,15.795,0.045
1,26-08-2025,15.890,15.925,0.043
2,26-08-2025,15.904,15.970,0.063
3,26-08-2025,16.018,16.090,0.069
4,26-08-2025,16.026,16.100,0.071
5,26-08-2025,16.053,16.130,0.074
6,26-08-2025,15.450,15.520,0.067
7,26-08-2025,13.896,13.896,0.000
8,26-08-2025,14.332,14.332,0.000
9,26-08-2025,14.774,14.774,0.000


In [20]:
input_df = all_weather_df.copy()

# input_df.loc[0, "generation_prev"] = electricity_df["Generation"].sum()
# input_df.loc[0, "loadshed_prev"] = electricity_df["Loadshed"].sum()
input_df

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77.0,30.0,34.1,26.4,2025,8,None,None
1,2025-08-27,0.0,9.746264,79.0,28.8,33.1,25.8,2025,8,None,None
2,2025-08-28,0.0,8.154183,83.0,28.0,31.0,25.8,2025,8,None,None
3,2025-08-29,0.0,10.073150,84.0,28.1,31.2,25.8,2025,8,None,None
4,2025-08-30,0.0,10.403442,84.0,28.3,31.4,25.7,2025,8,None,None
5,2025-08-31,0.0,11.292411,78.0,29.6,33.6,25.9,2025,8,None,None
6,2025-09-01,0.0,10.482981,79.0,29.8,33.9,26.5,2025,9,None,None


In [21]:
input_df.dtypes

Date               datetime64[ns]
Rainfall                  float64
Sunshine                  float64
Humidity                  float64
Temp_mean                 float64
Temp_max                  float64
Temp_min                  float64
Year                        int32
Month                       int32
loadshed_prev              object
generation_prev            object
dtype: object

# Load Model


In [46]:
import lightgbm as lgb

generation_model = lgb.Booster(model_file="generation_lgbm_model.txt")
loadshed_model = lgb.Booster(model_file="loadshed_lgbm_model.txt")

In [ ]:
# Set initial values
generation_previous = electricity_df["Generation"].sum()
loadshed_previous = electricity_df["Loadshed"].sum()

forecasts = []

for idx in range(len(input_df)):
    row = input_df.iloc[idx].to_frame().T.infer_objects()

    row["generation_prev"] = generation_previous
    row["loadshed_prev"] = loadshed_previous

    generation_pred = generation_model.predict(row.drop(columns="Date"))[0]
    generation_previous = generation_pred

    loadshed_pred = loadshed_model.predict(row.drop(columns="Date"))[0]
    loadshed_previous = loadshed_pred

    daily_forecast = {
        "date": row["Date"].astype(str).values[0],
        "generation_prediction": generation_pred,
        "loadshed_prediction": loadshed_pred,
    }
    forecasts.append(daily_forecast)

,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
0,2025-08-26,0.0,11.991942,77.0,30.0,34.1,26.4,2025,8,0.478,361.91


,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
1,2025-08-27,0.0,9.746264,79.0,28.8,33.1,25.8,2025,8,4.027571,193.837253


,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
2,2025-08-28,0.0,8.154183,83.0,28.0,31.0,25.8,2025,8,4.016455,195.96849


,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
3,2025-08-29,0.0,10.07315,84.0,28.1,31.2,25.8,2025,8,0.761906,190.197368


,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
4,2025-08-30,0.0,10.403442,84.0,28.3,31.4,25.7,2025,8,0.885554,196.016774


,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
5,2025-08-31,0.0,11.292411,78.0,29.6,33.6,25.9,2025,8,0.885554,195.348625


,Date,Rainfall,Sunshine,Humidity,Temp_mean,Temp_max,Temp_min,Year,Month,loadshed_prev,generation_prev
6,2025-09-01,0.0,10.482981,79.0,29.8,33.9,26.5,2025,9,4.025716,193.748152


In [48]:
forecasts

[{'date': '2025-08-26',
  'generation_prediction': 193.8372534576477,
  'loadshed_prediction': 4.0275706301534395},
 {'date': '2025-08-27',
  'generation_prediction': 195.96848965912665,
  'loadshed_prediction': 4.01645533683271},
 {'date': '2025-08-28',
  'generation_prediction': 190.19736822789773,
  'loadshed_prediction': 0.7619061483225922},
 {'date': '2025-08-29',
  'generation_prediction': 196.01677355397072,
  'loadshed_prediction': 0.8855544071569458},
 {'date': '2025-08-30',
  'generation_prediction': 195.34862498312188,
  'loadshed_prediction': 0.8855544071569458},
 {'date': '2025-08-31',
  'generation_prediction': 193.7481518045912,
  'loadshed_prediction': 4.0257159651166345},
 {'date': '2025-09-01',
  'generation_prediction': 196.17621113195864,
  'loadshed_prediction': 4.0257159651166345}]